In [1]:
import sys, os

sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
'''
观察本书源代码可知，上述代码在 mnist_show.py文件中。mnist_show.py文件的当前目录是 ch03，
但包含 load_mnist()函数的 mnist.py文件在 dataset目录下。因此，mnist_show.py文件不能跨目
录直接导入 mnist.py文件。sys.path.append(os.pardir)语句实际上是把父目录 deep-learning-
from-scratch加入到 sys.path（ Python 的搜索模块的路径集）中，从而可以导入 deep-learning-
from-scratch下的任何目录（包括 dataset目录）中的任何文件。
'''
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [2]:
from PIL import Image


def img_show(img):
    pil_img = Image.fromarray(img)
    pil_img.show()


img = x_train[0]
label = t_train[0]
print(label)  # 5

print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 把图像的形状变成原来的尺寸
print(img.shape)  # (28, 28)

img_show(img)

5
(784,)
(28, 28)


In [8]:
import pickle
import numpy as np


def get_data():
    _, (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)  # 获取概率最高的元素的索引
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


In [16]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [30]:
x, t = get_data()
network = init_network()

batch_size = 100  # 批数量
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i + batch_size]
    y_batch = predict(network, x_batch)
    '''
    这指定了在 100 × 10 的数组中，沿着第 1 维方向（以
    第 1 维为轴）找到值最大的元素的索引（第 0 维对应第 1 个维度）
    矩阵的第 0 维是列方向，第 1 维是行方向
    '''
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i + batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


In [26]:
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
# 返回沿轴axis最大值的索引, axis=1表示行方向
y = np.argmax(x, axis=1)
print(y)

[1 2 1 0]


In [24]:
y = np.array([1, 2, 1, 0])
t = np.array([1, 2, 0, 0])
print(y == t)
print(np.sum(y == t))
print(np.sum(y != t))

[ True  True False  True]
3
1
